<small>
Copyright (c) 2017 Andrew Glassner

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
</small>



# Deep Learning From Basics to Practice
## by Andrew Glassner, https://dlbasics.com, http://glassner.com
------
## Chapter 27: Applications
### Notebook 2: Content and Style Visualizations

This notebook is provided as a “behind-the-scenes” look at code used to make some of the figures in this chapter. It is still in the hacked-together form used to develop the figures, and is only lightly commented.

### About this code:
This notebook is adapted from
https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.3-neural-style-transfer.ipynb
by François Chollet.

See License C in LICENSE.txt

In [2]:
import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg16
from keras import backend as K_backend
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave  #  replace later
import time
import os

K_backend.set_image_data_format('channels_last')

Using TensorFlow backend.


In [3]:
# Because we will be making many images, and we don't
# want to fill up the notebook with one after another,
# we use imread and imsave from skimage.io instead 
# of the file_helper. 
#
# Make a File_Helper for saving and loading files.

save_files = True

import os, sys, inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.insert(0, os.path.dirname(current_dir)) # path to parent dir
from DLBasics_Utilities import File_Helper
file_helper = File_Helper(save_files)

# get the input and output directories
input_data_directory = file_helper.get_input_data_dir()
output_data_directory = file_helper.get_saved_output_dir()

# make sure the output directory exists
already_existed = file_helper.check_for_directory(output_data_directory)

In [4]:
def get_target_size(target_image_path, target_height):
    '''Return the size of the images we want to use. We use the same
    aspect ratio as the target, but with the given height.'''
    width, height = load_img(target_image_path).size
    img_width = int(width * target_height / height)
    return (img_width, target_height)

In [5]:
def preprocess_image(image_path, img_height, img_width):
    '''Process an image using VGG16 conventions'''
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

def deprocess_image(x):
    '''Undo VGG16 image conventions'''
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # change channel order from'BGR' to 'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [6]:
def get_VGG16(input_tensor):
    '''Get VGG16 model. Leave off the final fully-connected layers.
    Set it up with ImageNet weights. Set the input tensor to input_tensor.'''
    model = vgg16.VGG16(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
    return model

In [7]:
# Content loss

def content_loss(base, combination):
    '''The content loss wants the final layer of VGG16 to report
    the same conclusions for both the original target and the
    synthesized image.'''
    return K_backend.sum(K_backend.square(combination - base))

In [8]:
# Style loss

def gram_matrix(x):
    '''Compute a Gram matrix. Given a matrix F, element (i,j) of the Gram
    matrix is the dot product of rows i and j from matrix F.'''
    features = K_backend.batch_flatten(K_backend.permute_dimensions(x, (2, 0, 1)))
    gram = K_backend.dot(features, K_backend.transpose(features))
    return gram


def style_loss(style, combination, img_height, img_width):
    '''Compute the style loss. We want the Gram matrix for the original
    target and the synthetic image to be the same.'''
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K_backend.sum(K_backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [9]:
# Total variation loss

def total_variation_loss(x, img_height, img_width):
    '''Compute the total variation loss. This prevents neighboring pixels from
    becoming too different from one another.'''
    a = K_backend.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K_backend.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K_backend.sum(K_backend.pow(a + b, 1.25))

In [10]:
# An object to hold both loss and gradients, so we can calculate
# both at once and hang onto them, returning either one when asked.

class Evaluator(object):

    def __init__(self, img_height, img_width, fetch_loss_and_grads):
        self.loss_value = None
        self.grads_values = None
        self.img_height = img_height
        self.img_width = img_width
        self.fetch_loss_and_grads = fetch_loss_and_grads

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, self.img_height, self.img_width, 3))
        outs = self.fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [11]:
def make_synthetic_image(target_image_path, style_reference_image_path, output_prefix,
                        target_height, iterations, content_layer, style_layers,
                        total_variation_weight, style_weight, content_weight, random_seed):

    img_width, img_height = get_target_size(target_image_path, target_height)

    target_image = K_backend.constant(preprocess_image(target_image_path, img_height, img_width))
    style_reference_image = K_backend.constant(preprocess_image(style_reference_image_path, img_height, img_width))

    # This placeholder will contain our generated image
    # AG combination_image = K_backend.placeholder((1, img_height, img_width, 3))
    combination_image = K_backend.random_uniform_variable(shape=(1, img_height, img_width, 3), low=0, high=1)

    # We combine the 3 images into a single batch
    input_tensor = K_backend.concatenate([target_image,
                                  style_reference_image,
                                  combination_image], axis=0)

    model = get_VGG16(input_tensor)

    # Dict mapping layer names to activation tensors
    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])


    # Define the loss by adding all components to a `loss` variable
    loss = K_backend.variable(0.)
    layer_features = outputs_dict[content_layer]
    target_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss += content_weight * content_loss(target_image_features,
                                          combination_features)
    if len(style_layers) > 0:
        for layer_name in style_layers:
            layer_features = outputs_dict[layer_name]
            style_reference_features = layer_features[1, :, :, :]
            combination_features = layer_features[2, :, :, :]
            sl = style_loss(style_reference_features, combination_features, img_height, img_width)
            loss += (style_weight / len(style_layers)) * sl
        loss += total_variation_weight * total_variation_loss(combination_image, img_height, img_width)


    # Get the gradients of the generated image wrt the loss
    grads = K_backend.gradients(loss, combination_image)[0]

    # Function to fetch the values of the current loss and the current gradients
    fetch_loss_and_grads = K_backend.function([combination_image], [loss, grads])

    evaluator = Evaluator(img_height, img_width, fetch_loss_and_grads)

    # Run scipy-based optimization (L-BFGS) over the pixels of the generated image
    # so as to minimize the neural style loss.
    # This is our initial state: the target image.
    # Note that `scipy.optimize.fmin_l_bfgs_b` can only process flat vectors.
    x = preprocess_image(target_image_path, img_height, img_width)
    np.random.seed(random_seed)
    x = np.random.uniform(low=-128, high=128, size=x.shape)

    x = x.flatten()
    for i in range(iterations):
        print('Start of iteration', i)
        start_time = time.time()
        x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                         fprime=evaluator.grads, maxfun=20)
        print('Current loss value:', min_val)
        # Save current generated image
        img = x.copy().reshape((img_height, img_width, 3))
        img = deprocess_image(img)
        fname = output_prefix+'-iteration-{:d}-sw-{:.5f}-cw-{:.5f}-tvw-{:.6f}.png'.\
                                format(i, style_weight, content_weight, total_variation_weight)
        imsave(fname, img)
        end_time = time.time()
        print('Image saved as', fname)
        print('Iteration %d completed in %ds' % (i, end_time - start_time))
        


In [12]:
def make_style_visualizations():
    
    vgg16_layers_list = [
       'block1_conv1', 'block1_conv2',
       'block2_conv1', 'block2_conv2',
       'block3_conv1', 'block3_conv2', 'block3_conv3', 
       'block4_conv1', 'block4_conv2', 'block4_conv3', 
       'block5_conv1', 'block5_conv2', 'block5_conv3', 
    ]

    target_image_path = file_helper.get_input_file_path('waters-3038803_1280-crop.jpg')  # irrelevant for style viz
    style_reference_image_path = file_helper.get_input_file_path('HR-Self-Portrait-1907-Picasso.jpg')
    target_height = 200
    num_iterations = 50
    content_layer = 'block4_conv1'   # irrelevant for style
    total_variation_weight = 1e-3
    style_weight = 1
    content_weight = 0
    
    
    for i in range(len(vgg16_layers_list)):
        style_layers = vgg16_layers_list[:i+1]
        random_seed = 42+i
        output_prefix = output_data_directory+'/style-to-'+style_layers[-1]
        print("Starting style viz of layers ",style_layers)
        outputs = make_synthetic_image(target_image_path=target_image_path, 
                                       style_reference_image_path=style_reference_image_path, 
                                       output_prefix = output_prefix,
                                       target_height = target_height, 
                                       iterations = num_iterations,
                                       content_layer = content_layer,
                                       style_layers = style_layers,
                                       total_variation_weight = total_variation_weight,
                                       style_weight = style_weight,
                                       content_weight = content_weight,
                                       random_seed = random_seed)

In [23]:
def make_single_layer_style_visualizations():
    
    vgg16_layers_list = [
       'block1_conv1', 'block1_conv2',
       'block2_conv1', 'block2_conv2',
       'block3_conv1', 'block3_conv2', 'block3_conv3', 
       'block4_conv1', 'block4_conv2', 'block4_conv3', 
       'block5_conv1', 'block5_conv2', 'block5_conv3', 
    ]

    target_image_path = \
        file_helper.get_input_file_path('waters-3038803_1280-crop.jpg')  # irrelevant for style viz
    style_reference_image_path = \
        file_helper.get_input_file_path('HR-Self-Portrait-1907-Picasso.jpg')
    target_height = 200
    num_iterations = 50
    content_layer = 'block4_conv1'   # irrelevant for style
    total_variation_weight = 1e-3
    style_weight = 1
    content_weight = 0
    
    # we skip the first layer because we got it when we did the sets
    for i in range(1, len(vgg16_layers_list)):
        style_layers = [vgg16_layers_list[i]]
        random_seed = 424+i
        output_prefix = output_data_directory+'/style-only-'+style_layers[-1]
        print("Starting style viz of layers ",style_layers)
        outputs = make_synthetic_image(target_image_path=target_image_path, 
                                       style_reference_image_path=style_reference_image_path, 
                                       output_prefix = output_prefix,
                                       target_height = target_height, 
                                       iterations = num_iterations,
                                       content_layer = content_layer,
                                       style_layers = style_layers,
                                       total_variation_weight = total_variation_weight,
                                       style_weight = style_weight,
                                       content_weight = content_weight,
                                       random_seed = random_seed)

In [24]:
def make_content_visualizations():
    
    vgg16_layers_list = [
        'block1_conv1', 'block1_conv2',
        'block2_conv1', 'block2_conv2',
        'block3_conv1', 'block3_conv2', 'block3_conv3',
        'block4_conv1', 'block4_conv2', 'block4_conv3',
        'block5_conv1', 
        'block5_conv2', 'block5_conv3', 
    ]
        
    target_image_path = file_helper.get_input_file_path('waters-3038803_1280-crop.jpg')
    style_reference_image_path = file_helper.get_input_file_path('HR-Self-Portrait-1907-Picasso.jpg') # irrelevant for content viz
    style_layers = [] # irrelevant for content viz
    target_height = 200
    num_iterations = 50
    total_variation_weight = 1e-3
    style_weight = 0
    content_weight = 1
    
    for i in range(len(vgg16_layers_list)):
        content_layer = vgg16_layers_list[i]
        random_seed = 4242+i
        print("Starting content viz of layer ",content_layer)
        output_prefix = output_data_directory+'/content-'+content_layer
        outputs = make_synthetic_image(target_image_path=target_image_path, 
                                       style_reference_image_path=style_reference_image_path, 
                                       output_prefix = output_prefix,
                                       target_height = target_height, 
                                       iterations = num_iterations,
                                       content_layer = content_layer,
                                       style_layers = style_layers,
                                       total_variation_weight = total_variation_weight,
                                       style_weight = style_weight,
                                       content_weight = content_weight,
                                       random_seed = random_seed)

In [25]:
def make_style_and_content_visualizations():
    #make_style_visualizations()
    #make_content_visualizations()
    make_single_layer_style_visualizations()

In [26]:
make_style_and_content_visualizations()       

Starting style viz of layers  ['block1_conv2']
Start of iteration 0
Current loss value: 2.97738e+09
Image saved as saved_output/style-only-block1_conv2-iteration-0-sw-1.00000-cw-0.00000-tvw-0.001000.png
Iteration 0 completed in 60s
Start of iteration 1
Current loss value: 4.21604e+08
Image saved as saved_output/style-only-block1_conv2-iteration-1-sw-1.00000-cw-0.00000-tvw-0.001000.png
Iteration 1 completed in 62s
Start of iteration 2
Current loss value: 1.2334e+08
Image saved as saved_output/style-only-block1_conv2-iteration-2-sw-1.00000-cw-0.00000-tvw-0.001000.png
Iteration 2 completed in 61s
Start of iteration 3
Current loss value: 7.86937e+07
Image saved as saved_output/style-only-block1_conv2-iteration-3-sw-1.00000-cw-0.00000-tvw-0.001000.png
Iteration 3 completed in 62s
Start of iteration 4
Current loss value: 6.05925e+07
Image saved as saved_output/style-only-block1_conv2-iteration-4-sw-1.00000-cw-0.00000-tvw-0.001000.png
Iteration 4 completed in 66s
Start of iteration 5
Current 